# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [61]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier



from sklearn import tree
from sklearn.metrics import accuracy_score, classification_report

import pydotplus
from IPython.display import Image

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [45]:
# Create our features
binary_df = pd.get_dummies(df, columns=[ 'issue_d', 'initial_list_status', 'next_pymnt_d','application_type', 'home_ownership', 'verification_status', 'loan_status', 'pymnt_plan', 'hardship_flag', 'debt_settlement_flag'])

X = binary_df.drop(columns=['loan_status_high_risk', 'loan_status_low_risk'])

# Create our target
y = df.loan_status

In [46]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.009285,0.526309,0.106747,0.357659,0.478007,0.373992,0.148001,1.0,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.095914,0.499311,0.308793,0.479314,0.499520,0.483865,0.355104,0.0,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0


In [47]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [48]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [49]:
# Create the StandardScaler instance
data_scaler = StandardScaler()


In [51]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = data_scaler.fit(X_train)

In [52]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [55]:
# Resample the training data with the BalancedRandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)
rf_model = rf_model.fit(X_train_scaled, y_train)
rf_predictions = rf_model.predict(X_test_scaled)


In [56]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, rf_predictions)

0.683080618047773

In [57]:
# Display the confusion matrix
confusion_matrix(y_test, rf_predictions)

array([[   37,    64],
       [    3, 17101]], dtype=int64)

In [58]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, rf_predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.93      0.37      1.00      0.52      0.61      0.34       101
   low_risk       1.00      1.00      0.37      1.00      0.61      0.39     17104

avg / total       1.00      1.00      0.37      1.00      0.61      0.39     17205



In [59]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.07306479006287282, 'last_pymnt_amnt'),
 (0.06441359093730943, 'total_rec_prncp'),
 (0.06428366152112415, 'total_rec_int'),
 (0.06296170249911864, 'total_pymnt_inv'),
 (0.060136260046728646, 'total_pymnt'),
 (0.02145654676383674, 'out_prncp_inv'),
 (0.02076879589965165, 'installment'),
 (0.01987766445497464, 'out_prncp'),
 (0.01801374934457673, 'dti'),
 (0.016524335313657667, 'max_bal_bc'),
 (0.015375418266349531, 'mo_sin_old_rev_tl_op'),
 (0.01527472937512379, 'revol_bal'),
 (0.015077527924358687, 'loan_amnt'),
 (0.014951352321187272, 'total_bal_ex_mort'),
 (0.014764522054847603, 'mo_sin_old_il_acct'),
 (0.01417600768132073, 'tot_cur_bal'),
 (0.014123345041279272, 'tot_hi_cred_lim'),
 (0.01399479492248306, 'bc_open_to_buy'),
 (0.013943326754450742, 'total_bal_il'),
 (0.013605388425385544, 'total_il_high_credit_limit'),
 (0.013544972204981623, 'avg_cur_bal'),
 (0.013318873835442308, 'bc_util'),
 (0.013013677430647816, 'annual_inc'),
 (0.013001661611179311, 'total_bc_limit'),
 (0.012

### Easy Ensemble Classifier

In [62]:
# Train the Classifier
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(n_estimators=20,
                                            learning_rate=learning_rate,
                                            max_features=5,
                                            max_depth=3,
                                            random_state=0)

    # Fit the model
    classifier.fit(X_train_scaled, y_train.ravel())
    print("Learning rate: ", learning_rate)

    # Score the model
    print("Accuracy score (training): {0:.3f}".format(
        classifier.score(
            X_train_scaled,
            y_train.ravel())))
    print("Accuracy score (validation): {0:.3f}".format(
        classifier.score(
            X_test_scaled,
            y_test.ravel())))
    print()

Learning rate:  0.05
Accuracy score (training): 0.996
Accuracy score (validation): 0.995

Learning rate:  0.1
Accuracy score (training): 0.997
Accuracy score (validation): 0.995

Learning rate:  0.25
Accuracy score (training): 0.995
Accuracy score (validation): 0.994

Learning rate:  0.5
Accuracy score (training): 0.996
Accuracy score (validation): 0.995

Learning rate:  0.75
Accuracy score (training): 0.996
Accuracy score (validation): 0.995

Learning rate:  1
Accuracy score (training): 0.995
Accuracy score (validation): 0.993



In [63]:
classifier = GradientBoostingClassifier(n_estimators=20,
                                        learning_rate=0.1,
                                        max_features=5,
                                        max_depth=3,
                                        random_state=0)

# Fit the model
classifier.fit(X_train_scaled, y_train.ravel())

# Make Prediction
predictions = classifier.predict(X_test_scaled)

In [64]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, predictions)

0.628391309079458

In [65]:
# Display the confusion matrix
confusion_matrix(y_test, predictions)

array([[   26,    75],
       [   11, 17093]], dtype=int64)

In [66]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.70      0.26      1.00      0.38      0.51      0.24       101
   low_risk       1.00      1.00      0.26      1.00      0.51      0.28     17104

avg / total       0.99      1.00      0.26      0.99      0.51      0.28     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.

In [71]:
print('1. The Random Forest model had the best balanced accuracy score 2. Both models had low recall scores but the Random Forest model has a better recall score. 3. Random Forest has the best geometric mean score. 4. The top 3 features are Last Payment Amount, Total Rec. Principal and Total Rec. Integer.')

1. The Random Forest model had the best balanced accuracy score 2. Both models had low recall scores but the Random Forest model has a better recall score. 3. Random Forest has the best geometric mean score. 4. The top 3 features are Last Payment Amount, Total Rec. Principal and Total Rec. Integer.
